In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

d:\Python\CODE\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
google_embeddings.embed_query("Hello AI")

[0.01908188685774803,
 -0.04838583618402481,
 -0.0033970330841839314,
 -0.015432494692504406,
 0.026915451511740685,
 -0.037899527698755264,
 0.03683774173259735,
 0.0024586953222751617,
 0.031139591708779335,
 -0.002974053379148245,
 0.05297260358929634,
 0.027224618941545486,
 -0.03135930001735687,
 -0.03521350398659706,
 0.012696987949311733,
 -0.03398113697767258,
 -0.005759889259934425,
 0.04882349446415901,
 -0.025584226474165916,
 -0.016702925786376,
 0.03510289266705513,
 -0.0072209034115076065,
 -0.011071284301578999,
 -0.03638654947280884,
 0.02125084027647972,
 0.018635960295796394,
 0.022123489528894424,
 -0.053073909133672714,
 -0.03174823150038719,
 0.03385208174586296,
 -0.03267931193113327,
 0.05955952778458595,
 -0.04031715542078018,
 -0.01112906914204359,
 0.04654634743928909,
 -0.05826421454548836,
 0.03411427512764931,
 0.019681427627801895,
 0.008696145378053188,
 -0.004346166737377644,
 0.0029152834322303534,
 -0.08902806788682938,
 -0.015501082874834538,
 -0.0179

In [5]:
len(google_embeddings.embed_query("Hello AI"))

768

In [9]:
from pinecone import Pinecone, ServerlessSpec
pc=Pinecone(api_key=pinecone_api_key)

In [15]:
index_name = "agentic-ai"
if index_name not in pc.list_indexes():
	pc.create_index(
		name=index_name,
		dimension=768,
		metric='cosine',
		spec=ServerlessSpec(cloud='aws',region='us-east-1')
	)

In [16]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'agentic-ai-on18n34.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'agentic-ai',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [17]:
index = pc.Index(index_name)

In [18]:
from langchain_pinecone import PineconeVectorStore
vectore_store = PineconeVectorStore(index=index,embedding=google_embeddings)

In [20]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [30]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]

In [31]:
vectore_store.add_documents(documents=documents, ids=uuids)

['8a04b02d-7eb4-4953-bd44-bbf5b8b77e42',
 'ac03632c-58f1-4991-9c9a-9f8d5b62d6e3',
 '197f1827-7b24-455f-8c98-710d223d947f',
 '432361cd-8f14-4607-a03d-11ce2c651eb2',
 'ee44e2d7-906a-445f-9bff-991d31e755ab',
 '1df52249-b838-4d6d-b165-f243f4950264',
 'fa47ef8c-a524-4f51-bfc0-b8ac6f1562ec',
 'adba65bc-a426-4393-b241-806484db77c0',
 'cf8ea933-8cc7-49b1-ae46-dae8e7a6c91e',
 '8e732e6b-4885-4790-91c6-1bda10fb1774']

In [32]:
result = vectore_store.similarity_search("what is langchain?")
result

[Document(id='197f1827-7b24-455f-8c98-710d223d947f', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='adba65bc-a426-4393-b241-806484db77c0', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='8e732e6b-4885-4790-91c6-1bda10fb1774', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='ee44e2d7-906a-445f-9bff-991d31e755ab', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

In [34]:
result = vectore_store.similarity_search("what is langchain?",filter={"source":"tweet"},k=2)
result

[Document(id='197f1827-7b24-455f-8c98-710d223d947f', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='adba65bc-a426-4393-b241-806484db77c0', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [35]:
retriever = vectore_store.as_retriever(
    search_type="similarity_score_threshold",
	search_kwargs={"k":3,"score_threshold":0.5}
)


In [36]:
retriever.invoke("langchain")

[Document(id='197f1827-7b24-455f-8c98-710d223d947f', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='adba65bc-a426-4393-b241-806484db77c0', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='8e732e6b-4885-4790-91c6-1bda10fb1774', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [38]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

d:\Python\CODE\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [42]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('llama.pdf')
docs = loader.load()

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
split_docs = splitter.split_documents(docs)

In [44]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(split_docs))]

In [45]:
vectore_store.add_documents(documents=split_docs, ids=uuids)

['c908f37c-2297-49cb-98c0-9160413705d9',
 '3e566da4-4ed2-44fd-857e-96ea28c5f8c6',
 'a640c094-2a6e-4b4c-be87-1f602359c8ae',
 '4509ac55-84b8-403f-bf9d-35e69db7300a',
 '425b97db-526c-4464-a552-fe1d563d2e43',
 '74043558-1ff7-43a7-afda-302de470e1f2',
 'e2e57cba-5e29-4685-a107-2e1b990547bb',
 'ea73987f-7cda-4fa3-b44d-7e4efbb2127d',
 '99620c3b-8a82-4b20-be0d-9f6a03b38a8d',
 '6fe890f8-0c37-4a1e-9317-17ba778f0379',
 'c042f754-4d8e-4a51-a520-e9c7a60de89f',
 '9ccb1381-496f-404c-9f25-ccc94bf091c3',
 'ed4cd070-b808-47f0-8191-6ae71dcf3fc6',
 'eeefc1ae-d133-4996-8f8d-9400d66c544b',
 '46424077-2579-4591-9bdd-42b607deccd9',
 'd7c2d901-3c48-4ca6-837e-f5429d910de0',
 '9090c32a-c7b7-4dad-87e2-4e722213840e',
 '539f1a3e-d7da-42d6-ae37-4d5ebeeada22',
 'e95e0cc5-fd6f-40d9-8209-9f42f476adc1',
 'c1c73b60-e0ea-461d-bb18-0b16e5966430',
 '55bc264e-b19f-4beb-aad1-c157478dae29',
 '5012a177-0d48-4e67-b1d8-0af93846b1ee',
 '0d29c43c-e2e5-49a3-9fff-f6e3e8d37e09',
 '0ad4c6f0-f1a3-4c38-ad40-bf3c027bb53e',
 '77f01518-ae9b-

In [46]:
chain = (
    {"context": retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [47]:
chain.invoke("what does llama do?")

'Llama 2 is a family of pretrained and fine-tuned large language models (LLMs). It carries risks with use and may produce inaccurate or objectionable responses. Developers should perform safety testing and tuning before deploying applications of Llama 2.'